In [ ]:
from deap import base, creator, tools, algorithms
import random
import numpy as np
import cv2

def correct_illumination(image, alpha=1.2, beta=20):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def filter_noise(image, kernel_size):
    if kernel_size <= 0:
        kernel_size = 1  # o cualquier otro número impar mayor que cero
    elif kernel_size % 2 == 0:
        kernel_size += 1  # hacerlo impar
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)


def enhance_contrast(image, clipLimit=3.0, tileGridSize=(10, 10)):
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    channels = cv2.split(image)
    clahe_channels = [clahe.apply(ch) for ch in channels]
    return cv2.merge(clahe_channels)

def detect_edges(image, low_threshold=30, high_threshold=90):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Canny(gray, low_threshold, high_threshold)

def additional_smoothing(image, kernel_size):
    if kernel_size <= 0:
        kernel_size = 1  # o cualquier otro número impar mayor que cero
    elif kernel_size % 2 == 0:
        kernel_size += 1  # hacerlo impar
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)


def threshold_segmentation(image, lower_bound, upper_bound):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Asegurarse de que lower_bound y upper_bound sean del mismo tipo de datos
    lower_bound = np.array(lower_bound, dtype=np.uint8)
    upper_bound = np.array(upper_bound, dtype=np.uint8)
    
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    return mask


def jaccard_index(mask, reference_mask):
    intersection = np.logical_and(mask, reference_mask)
    union = np.logical_or(mask, reference_mask)
    return np.sum(intersection) / np.sum(union)

def random_odd(min_val, max_val):
    return random.choice([x for x in range(min_val, max_val + 1) if x % 2 == 1])

def custom_mutate(individual, mu, sigma, indpb):
    tools.mutGaussian(individual, mu, sigma, indpb)
    # Asegurarse de que kernel_size_smooth sea impar y mayor que cero
    if individual[7] <= 0:
        individual[7] = 1  # o cualquier otro número impar mayor que cero
    elif individual[7] % 2 == 0:
        individual[7] += 1  # hacerlo impar



# Funciones de procesamiento de imágenes
# (Las mismas que en los ejemplos anteriores, pero omitidas aquí para ahorrar espacio)

# Función de evaluación para el algoritmo genético
def evaluate(individual):
    alpha, beta, kernel_size_noise, clipLimit, tileGridSize, low_threshold, high_threshold, kernel_size_smooth, lower_h, upper_h = individual

    print(f"Evaluating with kernel_size_noise: {kernel_size_noise}")  # Debugging line

    tileGridSize = int(round(tileGridSize))

    kernel_size_noise = int(round(kernel_size_noise))
    kernel_size_noise = kernel_size_noise if kernel_size_noise % 2 == 1 else kernel_size_noise + 1

    kernel_size_smooth = int(round(kernel_size_smooth))
    kernel_size_smooth = kernel_size_smooth if kernel_size_smooth % 2 == 1 else kernel_size_smooth + 1

    # Aplicar métodos de procesamiento de imágenes
    corrected_image = correct_illumination(image, alpha, beta)
    filtered_image = filter_noise(corrected_image, kernel_size_noise)
    enhanced_image = enhance_contrast(filtered_image, clipLimit, (tileGridSize, tileGridSize))
    edges = detect_edges(enhanced_image, low_threshold, high_threshold)
    print("Debugging kernel_size_smooth:", kernel_size_smooth)
    smoothed_edges = additional_smoothing(edges, kernel_size_smooth)

    # Segmentación por umbral
    lower_bound = np.array([lower_h, 100, 100])
    upper_bound = np.array([upper_h, 255, 255])
    mask = threshold_segmentation(enhanced_image, lower_bound, upper_bound)

    # Calcular el coeficiente de Jaccard
    jaccard = jaccard_index(mask, reference_mask)
    
    return jaccard,


if __name__ == "__main__":
    # Cargar la imagen y la máscara de referencia
    image = cv2.imread(R"C:\Users\Kevin\Documents\train\images\0070.png")
    reference_mask = cv2.imread(R"C:\Users\Kevin\Documents\train\labels\0070.png", cv2.IMREAD_GRAYSCALE)

    # Configuración del algoritmo genético
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    # Rangos de los parámetros ajustados
    toolbox.register("attr_float", random.uniform, 0.8, 1.2)  # alpha
    toolbox.register("attr_int", random.randint, 10, 30)  # beta
    toolbox.register("attr_odd", random_odd, 5, 9)  # kernel_size_noise
    toolbox.register("attr_float", random.uniform, 2.0, 4.0)  # clipLimit
    toolbox.register("attr_int", random.randint, 8, 12)  # tileGridSize
    toolbox.register("attr_int", random.randint, 40, 80)  # low_threshold
    toolbox.register("attr_int", random.randint, 100, 150)  # high_threshold
    toolbox.register("attr_odd_smooth", random_odd, 3, 11)  # kernel_size_smooth
    toolbox.register("attr_int", random.randint, 0, 5)  # lower_h
    toolbox.register("attr_int", random.randint, 10, 15)  # upper_h
    toolbox.register("mutate", custom_mutate, mu=0, sigma=1, indpb=0.2)
    toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.attr_float, toolbox.attr_int, toolbox.attr_odd, toolbox.attr_float, toolbox.attr_int, toolbox.attr_int, toolbox.attr_int, toolbox.attr_odd_smooth, toolbox.attr_int, toolbox.attr_int), 
                 n=1)

    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate)
    
    # Tamaño de la población ajustado
    population = toolbox.population(n=200)

    # Número de generaciones ajustado
    ngen = 100

    # Probabilidades de cruzamiento y mutación ajustadas
    probab_crossing, probab_mutating = 0.7, 0.3

    # Ejecución del algoritmo genético
    algorithms.eaSimple(population, toolbox, probab_crossing, probab_mutating, ngen, stats=None, halloffame=None, verbose=True)

    # Probabilidad de apareamiento y mutación
    probab_crossing, probab_mutating = 0.5, 0.2
            
